# Sistem Klasifikasi Anjing dan Kucing menggunakan Metode HOG

Coded By : Fauzan Firdaus (1301164317) & Nur Azizah Agustina (1301164076)

Pertama-tama, lakukan import terlebih dahulu untuk beberapa library yang akan digunakan.

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

from skimage.transform import resize
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import glob

Definisikan fungsi 'extractHOG' untuk melakukan proses ekstraksi fitur dengan metode HOG.

In [2]:
def extractHOG(img):
    ftr,_=hog(img, orientations=8, pixels_per_cell=(10, 10),
            cells_per_block=(1, 1), visualize=True, multichannel=False)
    return ftr

Setelah itu, fungsi 'featureExtraction' dibawah ini berfungsi untuk melakukan proses ekstraksi fitur bentuk secara iteratif untuk semua gambar dalam array.

In [3]:
def featureExtraction(arr):
    arr_feature=[]
    for i in range(np.shape(arr)[0]):
        arr_feature.append(extractHOG(arr[i]))
    return arr_feature

Fungsi selanjutnya adalah 'preprocessing' yang berfungsi untuk melakukan tahap preprocessing (membuat gambar menjadi abu-abu dan meresize gambar) agar proses komputasi lebih cepat.

In [4]:
def preprocessing(arr):
    arr_prep=[]
    for i in range(np.shape(arr)[0]):
        img=cv2.cvtColor(arr[i], cv2.COLOR_BGR2GRAY)
        img=resize(img, (72, 72),anti_aliasing=True)
        arr_prep.append(img)
    return arr_prep

Fungsi yang terakhir adalah 'loadData' yang berfungsi untuk memuat semua gambar/dataset dan labelnya menjadi bentuk array yang berada dalam folder Datasets.

In [5]:
def loadData(arr,labelobject):
    label=[]
    strr = "Datasets/"+str(labelobject)+"/*.jpg"
    for file in glob.glob(strr):
        img=np.asarray(plt.imread(file))
        arr.append(img)
        label.append(labelobject)
    return arr,label

Lakukan proses load gambar dan labelnya

In [6]:
cat=[]
dog=[]

cat,label_cat = loadData(cat,1)
dog,label_dog = loadData(dog,2)

Gabungkan semua dataset menjadi 1 array

In [7]:
combineImage = np.concatenate((cat,dog))
combineLabel = np.concatenate((label_cat,label_dog))

Pecah dataset menjadi 2 bagian, yaitu data train dan data test. Hal ini dilakukan bertujuan dalam proses memvalidasi dataset.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(combineImage, combineLabel, test_size=0.2, random_state=42)

Lakukan preprocessing terlebih dahulu

In [9]:
X_trainp=preprocessing(X_train)
X_testp=preprocessing(X_test)

/Users/faoezanf/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Lalu, lakukan proses ekstraksi.

In [10]:
X_trainext=featureExtraction(X_trainp)
X_testext=featureExtraction(X_testp)

/Users/faoezanf/anaconda3/lib/python3.7/site-packages/skimage/feature/_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)


Tahap akhir adalah melakukan learning (fit) model / data train. Setelah melakukan learning, proses yang dilakukan adalah melakukan prediksi label, lalu muncullah akurasi sebagai ketepatan dari prediksi label tersebut dengan label yang sebenarnya.

In [11]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_trainext, y_train)
result = clf.predict(X_testext)
print("Akurasi HOG :", accuracy_score(y_test, result) * 100)

Akurasi HOG : 64.75
